In [19]:
using Random
using LinearAlgebra
using Statistics
using StatsBase
using Plots
using Base.Threads
using Flux
using Base.Iterators: product
using Flux: Optimise
using Flux: ADAM, params, update!
using Serialization

In [7]:
include("permanents.jl")

# Variables are for reward_function.jl
Lambda = 0.35 # Weight for regularizing the reward function to generate more ones (too high a labda will result in higher odds of generating isosceles triangles)

# Variables are for bitwise_model.jl
n_actions = 2  # Number of actions that the agent can take. In this case, it is either 0 for excluding a point and 1 for including it
n_sessions = 2000  # Number of new sessions per iteration
learning_rate = 0.001  # Learning rate, increase this to converge faster
percentile = 90  # Top 100-x percentile the agent will learn from
super_percentile = 90  # Top 100-x percentile of that survives to the next generation

90

In [60]:
# Variables are for bitwise_model.jl
n_actions = 2  # Number of actions that the agent can take. In this case, it is either 0 for excluding a point and 1 for including it
n_sessions = 2000  # Number of new sessions per iteration
learning_rate = 0.001  # Learning rate, increase this to converge faster
percentile = 90  # Top 100-x percentile the agent will learn from
super_percentile = 90  # Top 100-x percentile of that survives to the next generation

90

In [38]:
states = zeros(Float32, n_sessions, observation_space, len_game)
actions = zeros(Int, n_sessions, len_game)
state_next = zeros(Float32, n_sessions, observation_space)
states[:, len_word, 1]
step_ = 0;

# Helper functions

In [ ]:
convert(Array{Float64}, a)

In [372]:
reshape(state_example,(n,n))'

4×4 adjoint(::Matrix{Float64}) with eltype Float64:
 1.0  1.0  0.0  0.0
 1.0  1.0  1.0  1.0
 0.0  1.0  1.0  1.0
 1.0  1.0  0.0  1.0

In [448]:
file_name = joinpath("Data","test_file.txt")
open(file_name, "a") do f
    write(f, re, "\n")
end

11

In [480]:
if !isdir("Data")
    mkdir("Data")
end
file_name = joinpath("Data","test_file.txt")
open(joinpath("Data","test_file.txt"), "a") do f
    write(f, example_string, '\n')
end

57

In [456]:
joinpath("data","test"*".txt")

"data/test.txt"

In [460]:
pre = reshape(convert(Vector{Int}, state_example),(n,n))

4×4 Matrix{Int64}:
 1  1  0  1
 1  1  1  1
 0  1  1  0
 0  1  1  1

In [483]:
function board_to_string(board, n)
    # board is currently a Vector
    board = reshape(convert(Vector{Int}, board), (n,n))
    
    output = "["
    for i in 1:n-1
        output = output * string(board[:,i]) * '\n' * ' '
    end
    output = output * string(board[:,n]) * "]"

    return output
end

board_to_string (generic function with 2 methods)

In [474]:
example_string = board_to_string(state_example, n)

"[[1, 1, 0, 0]\n [1, 1, 1, 1]\n [0, 1, 1, 1]\n [1, 1, 0, 1]]"

In [424]:
test = "["
for row in 1:n-1 
    test = test * string(pre[:,row]) * '\n' * ' '
end
test = test * string(pre[:,n]) * "]"

"[[1, 1, 0, 0]\n [1, 1, 1, 1]\n [0, 1, 1, 1]\n [1, 1, 0, 1]]"

In [451]:
reshape(state_example,(n,n))

4×4 Matrix{Float64}:
 1.0  1.0  0.0  1.0
 1.0  1.0  1.0  1.0
 0.0  1.0  1.0  0.0
 0.0  1.0  1.0  1.0

In [401]:
test = string(pre[:,1]) * '\n' * string(pre[:,2])

"[1, 1, 0, 0]\n[1, 1, 1, 1]"

In [458]:
state_example

16-element Vector{Float64}:
 1.0
 1.0
 0.0
 0.0
 1.0
 1.0
 1.0
 1.0
 0.0
 1.0
 1.0
 1.0
 1.0
 1.0
 0.0
 1.0

In [240]:
function new_point_allowed(one_indices, new_point_index, n)
    row = ceil(Int, new_point_index / n)
    col = new_point_index % n
    if col == 0; col = n end # julia is 1-indexed, so x*n mod(n) must be n rather than 0

    point_allowed = true

    for i in 1:length(one_indices) 
        for j in i+1:length(one_indices)
            point_one_row = ceil(Int, one_indices[i] / n)
            point_two_row = ceil(Int, one_indices[j] / n)
            #if point_one_row == 0; point_one_row end
            #if point_two_row == 0; point_two_row end

            point_one_col = one_indices[i] % n
            point_two_col = one_indices[j] % n
            if point_one_col == 0; point_one_col = n end
            if point_two_col == 0; point_two_col = n end

            if (  (row == point_one_row)
                | (row == point_two_row)
                | (point_one_row == point_two_row)
                | (col == point_one_col)
                | (col == point_two_col)
                | (point_one_col == point_two_col))
                continue
            end
            
            # ensure point_one_col < point_two_col
            if (point_two_col < point_one_col)
                point_one_row, point_two_row = point_two_row, point_one_row
                point_one_col, point_two_col = point_two_col, point_one_col
            end

            # new point as 1 in a valid 312-pattern
            if ((point_two_row < point_one_row < row) & (point_one_col < col < point_two_col))
                point_allowed = false
                break
            end

            # new point as 2 in a valid 312-pattern
            if ((point_two_row < row < point_one_row) & (col < point_one_col))
                point_allowed = false
                break
            end

            # new point as 3 in a valid 312-pattern
            if ((row < point_one_row < point_two_row) & (point_two_col < col))
                point_allowed = false
                break
            end
        end

        if point_allowed == false
            break
        end
        
    end

    return point_allowed
end

new_point_allowed (generic function with 1 method)

In [241]:
function add_point(input_state, action_vec, n)
    point_added = false
    action_taken = zeros(n^2)
    cur_state = copy(input_state)
    
    while !point_added
        action_index = StatsBase.sample(collect(1:length(action_vec)), Weights(action_vec), 1)[1]
        one_indices = findall(!iszero, input_state)
        action_allowed = new_point_allowed(one_indices, action_index, n)
        
        if (cur_state[action_index] == 0) & (action_allowed)
            cur_state[action_index] = 1
            action_taken[action_index] = 1
            point_added = true
        else
            action_vec[action_index] = 0
            action_vec = action_vec ./ sum(action_vec)
        end
    end

    return cur_state, action_taken
end

add_point (generic function with 1 method)

In [291]:
n_sessions = 3
n = 4

i = 1

states = zeros(n_sessions, 4*n - 4 + 1, n^2)
actions = zeros(n_sessions, 4*n - 4 + 1, n^2)
scores = zeros(n_sessions)

for i in 1:n_sessions
    step_ = 0
    while step_ < 4*n - 4
        step_ += 1
        cur_state = states[i, step_, :]

        output = net(cur_state)
        next_state, action = add_point(cur_state, output, n)

        actions[i, step_, :] = action
        states[i, step_ + 1, :] = next_state
    end
    final_state = states[i, step_ + 1, :]
    final_state_matrix = reshape(final_state, (n,n))
    scores[i] = glynn(final_state_matrix)
end

M = reshape(states[1,4*n - 4 + 1,:], (n,n))'

4×4 adjoint(::Matrix{Float64}) with eltype Float64:
 1.0  1.0  0.0  0.0
 1.0  1.0  1.0  1.0
 0.0  1.0  1.0  1.0
 1.0  1.0  0.0  1.0

In [295]:
states_batch, actions_batch, rewards_batch = states, actions,scores
sessions = (states_batch, actions_batch, rewards_batch)
reshape(states_batch[1,13,:],(4,4))'

4×4 adjoint(::Matrix{Float64}) with eltype Float64:
 1.0  1.0  0.0  0.0
 1.0  1.0  1.0  1.0
 0.0  1.0  1.0  1.0
 1.0  1.0  0.0  1.0

In [351]:
cat(rewards_batch, [4]; dims=1)

4-element Vector{Float64}:
 7.0
 6.0
 6.0
 4.0

In [353]:
super_sessions = [[sessions[1][i,:,:], sessions[2][i,:,:], sessions[3][i]] for i in 1:length(sessions[3])]
sort!(super_sessions, by = x -> x[3], rev=true)

3-element Vector{Vector{Any}}:
 [[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 1.0 1.0 … 0.0 1.0; 1.0 1.0 … 0.0 1.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], 7.0]
 [[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 1.0 … 0.0 1.0; 1.0 1.0 … 0.0 1.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 1.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], 6.0]
 [[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 1.0 0.0 … 0.0 1.0; 1.0 1.0 … 0.0 1.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], 6.0]

In [357]:
super_sessions[1][3]

6.0

In [356]:
sessions[3]

3-element Vector{Float64}:
 7.0
 6.0
 6.0

In [359]:
super_rewards

3-element Vector{Float64}:
 7.0
 6.0
 6.0

In [358]:
super_states = [super_sessions[i][1] for i in 1:length(super_sessions)]
super_actions = [super_sessions[i][2] for i in 1:length(super_sessions)]
super_rewards = [super_sessions[i][3] for i in 1:length(super_sessions)]

3-element Vector{Float64}:
 7.0
 6.0
 6.0

In [368]:
state_example = super_states[1][13,:]

16-element Vector{Float64}:
 1.0
 1.0
 0.0
 0.0
 1.0
 1.0
 1.0
 1.0
 0.0
 1.0
 1.0
 1.0
 1.0
 1.0
 0.0
 1.0

In [302]:
sessions[1][1,13,:]

16-element Vector{Float64}:
 1.0
 1.0
 0.0
 0.0
 1.0
 1.0
 1.0
 1.0
 0.0
 1.0
 1.0
 1.0
 1.0
 1.0
 0.0
 1.0

In [153]:
step_ = 4
i = 1
cur_state = states[i, step_, :]

output = net(cur_state)
next_state, action = add_point(cur_state, output, n)

([1.0, 1.0, 1.0, 1.0], [1.0, 0.0, 0.0, 0.0])

In [154]:
actions[i, step_, :] = action
states[i, step_ + 1, :] = next_state

4-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0

In [155]:
states[1,:,:]

5×4 Matrix{Float64}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 0.0  1.0  0.0  1.0
 0.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0

In [567]:
temp = reshape(item, 1, size(states_batch, 3)) # size of board
#elite_states = vcat(elite_states, temp)

LoadError: UndefVarError: `item` not defined

In [529]:
reshape(ex_states[1,:,:],1,size(states_batch,3))

LoadError: DimensionMismatch: new dimensions (1, 16) must be consistent with array size 208

In [574]:
concat = reshape(ex_states[2,:,:], (1,size(ex_states[2,:,:])...))
#concat[1,:,:]
test = cat(ex_states,concat;dims=1)

4×13×16 Array{Float64, 3}:
[:, :, 1] =
 0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0

[:, :, 3] =
 0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

;;; … 

[:, :, 14] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.

In [557]:
ex_states[2,:,:]

13×16 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     1.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     1.0  0.0  1.0  0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  …  1.0  0.0  1.0  1.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     1.0  0.0  1.0  1.0  0.0  1.0  1.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     1.0  0.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0     1.0  0.0  1.0  1.0  1.0  1.0  1.0
 1.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0     1.0  0.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  0.0  0.0  0.0  1.0  0.0  …  1.0  0.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  0.0  0.0  0.0  1.0  0.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  0.0  0.0  1.0  1.0  0.0  

In [570]:
concat=cat(ex_states,ex_states[2,:,:];dims=3)

LoadError: DimensionMismatch: mismatch in dimension 1 (expected 3 got 13)

In [531]:
ex_states[1,:,:]

13×16 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  …  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  1.0  0.0  1.0  0.0  0.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  1.0  0.0  1.0  1.0  0.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  1.0  0.0  1.0  1.0  1.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  1.0  0.0  1.0  1.0  1.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  1.0  0.0  1.0  1.0  1.0  …  1.0  0.0  0.0  1.0  0.0  0.0  0.0
 1.0  1.0  1.0  1.0  0.0  1.0  1.0  1.0     1.0  0.0  1.0  1.0  0.0  0.0  0.0
 1.0  1.0  1.0  1.0  0.0  1.0  1.0  1.0  

In [520]:
ex_states, ex_actions, ex_scores = generate_session(net, 3, 4)
ex_states

3×13×16 Array{Float64, 3}:
[:, :, 1] =
 0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0

[:, :, 3] =
 0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

;;; … 

[:, :, 14] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 15] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [644]:
function generate_session(agent, n_sessions, n)
    states = zeros(n_sessions, 4*n - 4 + 1, n^2)
    actions = zeros(n_sessions, 4*n - 4 + 1, n^2)
    scores = zeros(n_sessions)

    step = 0

    for i in 1:n_sessions # [1, n_sessions] inclusive
        step = 0
        while step < 4*n - 4 # there are 4*n - 4 steps total, starts at 0 and ends at 4*n - 4 - 1
            
            step += 1
            # current board
            cur_state .= states[i, step, :]

            output = agent(cur_state)

            next_state, action = add_point(cur_state, output, n)

            actions[i, step, :] = action
            states[i, step + 1, :] = next_state

        final_state = states[i, step + 1, :]
        final_state_matrix = reshape(final_state, (n,n))
        scores[i] = glynn(final_state_matrix)
        end
    end

    return states, actions, scores
end

generate_session (generic function with 1 method)

In [650]:
function select_super_sessions(states_batch, actions_batch, rewards_batch, super_percentile)
    counter = n_sessions * (100 - super_percentile) / 100
    println(counter)
    reward_threshold = quantile(rewards_batch, super_percentile / 100)
    
    super_states = [] #zeros(Float32, 0, size(states_batch, 2), size(states_batch, 3))
    super_actions = [] #Matrix{Int}(undef, 0, size(actions_batch, 2))
    super_rewards = [] #Vector{Float32}(undef, 0)

    for i in 1:size(states_batch, 1)
        if rewards_batch[i] >= reward_threshold - 0.000001 && counter > 0
            super_states = cat(super_states, states_batch[i, :, :]; dims=1)
            super_actions = cat(super_actions, states_batch[i, :, :]; dims=1)
            println("after cat")
            push!(super_rewards, rewards_batch[i])
        end
        counter -= 1
    end

    return super_states, super_actions, super_rewards

end

select_super_sessions (generic function with 1 method)

In [639]:
function select_elites(states_batch, actions_batch, rewards_batch, percentile)
    counter = n_sessions * (100 - percentile) / 100
    reward_threshold = quantile(rewards_batch, percentile / 100)

    elite_states = Matrix{Float32}(undef, 0, size(states_batch, 3))
    elite_actions = Matrix{Float32}(undef, 0, size(actions_batch, 3))

    for i in 1:size(states_batch, 1)
        if rewards_batch[i] >= reward_threshold - 0.000001 && counter > 0
            #for item in eachrow(states_batch[i, :, :])
            #    temp_state = reshape(item)
            #end
            
            for item in eachrow(states_batch[i, :, :])
                temp_state = reshape(item, 1, size(states_batch, 3)) # size of board
                elite_states = vcat(elite_states, temp_state)
            end
            #temp_state = reshape(states_batch[i, :, :], (1,size(states_batch[i, :, :])...))
            #elite_states = cat(elite_states, temp_state; dims=1)
            for item in eachrow(actions_batch[i, :, :])
                temp_action = reshape(item, 1, size(actions_batch, 3))
                elite_actions = vcat(elite_actions, temp_action)
                #push!(elite_actions, item)
            end
            #println(actions_batch[i])
            #for item in actions_batch[i, :]
            #    push!(elite_actions, item)
            #end
            #temp_actions = reshape(actions_batch[i, :, :], (1,size(actions_batch[i, :, :])...))
            #elite_actions = cat(elite_actions, temp_actions; dims=1)
        end
        counter -= 1
    end

    return elite_states, elite_actions
end

select_elites (generic function with 1 method)

In [648]:
function train(board_size, filename)
    n_sessions = 100
    Learning_rate = 0.0001

    n = board_size
    input_space = n*n

    first_layer_neurons = 128
    second_layer_neurons = 64
    third_layers_neurons = 4

    # Define the neural network architecture (similar to PyTorch)
    model = Chain(
        Dense(n^2, first_layer_neurons, relu),
        Dense(first_layer_neurons, second_layer_neurons, relu),
        Dense(second_layer_neurons, third_layers_neurons, relu),
        Dense(third_layers_neurons, n^2, σ)
    )

    # Create an instance of the neural network
    net = model

    # Defining the loss function and optimizer (similar to PyTorch)
    criterion(y_pred, y_true) = Flux.binarycrossentropy(y_pred, y_true)
    optimizer = Optimise.ADAM(learning_rate)

    # Global lists
    global super_states = []
    global super_actions = []
    global super_rewards = []

    cur_best_reward = 0
    cur_best_board = []
    cur_best_game = []

    for i in 1:10
        println("\n GENERATION $i")
        states_batch, actions_batch, rewards_batch = generate_session(net, n_sessions, n)

        if i > 1
            states_batch = cat(states_batch, super_states; dim=1)
            actions_batch = cat(actions_batch, super_actions; dim=1)
            rewards_batch = cat(rewards_batch, super_rewards; dim=1)
        end
        println(i)
        elite_states, elite_actions = select_elites(states_batch, actions_batch, rewards_batch, percentile)

        # sessions[1][i,:,:] are the states corresponding to the ith session
        # likewise for actions and rewards
        # the outer dimension is now the number of sessions
        # the next dimension indicates (1 = states, 2 = actions, 3 = rewards)
        # reverse sort the sessions (in the outer dimension) based on the rewards
        println("after elites")
        sessions = select_super_sessions(states_batch, actions_batch, rewards_batch, super_percentile)
        println("after select super")
        super_sessions = [[sessions[1][i,:,:], sessions[2][i,:,:], sessions[3][i]] for i in 1:length(sessions[3])]
        sort!(super_sessions, by = x -> x[3], rev=true)
        println("after sort")
        # optimize
        # Backward pass (gradients calculation) and optimization (similar to PyTorch)
        outputs = zeros(Float32, size(elite_states, 1), 1)
        for i in 1:size(elite_states,1)
            outputs[i] = model(elite_states[i, :, :])[1]
        end
        println("outputs")

        println(elite_actions)
        loss = criterion(outputs, elite_actions)
        grads = gradient(() -> loss, params(model))
        Optimise.update!(optimizer, params(model), grads)

        # retrieve states, actions, rewards but sorted by rewards
        super_states = [super_sessions[i][1] for i in 1:length(super_sessions)]
        super_actions = [super_sessions[i][2] for i in 1:length(super_sessions)]
        super_rewards = [super_sessions[i][3] for i in 1:length(super_sessions)]

        #########
        println(super_rewards)
        println(sessions)
        #mean_all_reward = mean(rewards_batch[end-200:end])
        mean_best_reward = mean(super_rewards)

        println("\n$i. Best individuals: ", super_rewards)
        # Uncomment the line below to print out the mean best reward
        println("Mean best reward: $mean_best_reward")

        # Make a new folder if 'Data' folder does not exist
        if !isdir("Data")
            mkdir("Data")
        end

        #max_index = argmax(super_rewards)
        max_index = 1

        if super_rewards[max_index] > cur_best_reward
            cur_best_reward = super_rewards[max_index]
            cur_best_board = super_states[max_index][4*n-4,:] # best board as vector
            cur_best_game = super_states[max_index]

            best_states_set = Set()
            push!(best_states_set, string(cur_best_board))

            # add to file
            open(joinpath("Data", filename * "_best_board_timeline.txt"), "a") do f
                write(f, board_to_string(cur_best_board), '\n')
            end
            open(joinpath("Data", filename * "_best_reward_timeline.txt"), "a") do f
                write(f, string(cur_best_reward), '\n')
            end
        end

        if super_rewards[max_index] == cur_best_reward
            cur_best_board = super_states[max_index][4*n-4,:] # best board as vector
            if !in(string(cur_best_board), best_states_set)
                push!(best_states_set, string(cur_best_board))

                # add to file
                open(joinpath("Data", filename * "_best_board_timeline.txt"), "a") do f
                    write(f, board_to_string(cur_best_board), '\n')
                end
                open(joinpath("Data", filename * "_best_reward_timeline.txt"), "a") do f
                    write(f, string(cur_best_reward), '\n')
                end
            end
    
        end
        
    end
        println(cur_best_reward)
        return net, cur_best_game
end

train (generic function with 1 method)

In [649]:
n_sessions = 100
n = 4
filename = "4x4_test"
best_net, best_game = train(n, filename)


 GENERATION 1
1
after elites
10.0
after cat
after select super
after sort
outputs
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.

LoadError: DimensionMismatch: loss function expects size(ŷ) = (91, 1) to match size(y) = (91, 16)

# Execute training

In [516]:
n = 4
first_layer_neurons = 128
second_layer_neurons = 64
third_layers_neurons = 4

# Define the neural network architecture (similar to PyTorch)
model = Chain(
    Dense(n^2, first_layer_neurons, relu),
    Dense(first_layer_neurons, second_layer_neurons, relu),
    Dense(second_layer_neurons, third_layers_neurons, relu),
    Dense(third_layers_neurons, n^2, σ)
)

# Create an instance of the neural network
net = model

# Defining the loss function and optimizer (similar to PyTorch)
criterion(y_pred, y_true) = Flux.binarycrossentropy(y_pred, y_true)
optimizer = Optimise.ADAM(learning_rate)

Adam(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

# Scratch

In [48]:
i = 2
step_ = 0

0

In [54]:
action = 1
step_ += 1
state_next[1, step_] = 1
state_next[i, len_word + step_] = 0
state_next[i, len_word + step_ + 1] = 1
state_next

5×8 Matrix{Float32}:
 1.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [55]:
states[i, :, step_ + 1] .= state_next[i, :]

8-element view(::Array{Float32, 3}, 2, :, 4) with eltype Float32:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0

In [56]:
states

5×8×4 Array{Float32, 3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 1.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 4] =
 1.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [59]:
state_next[1,:]

8-element Vector{Float32}:
 1.0
 1.0
 1.0
 0.0
 0.0
 0.0
 0.0
 1.0